In [1]:
import gym
import numpy as np
import copy, random, os, subprocess, cv2
import tensorflow as tf
import keras.backend as K
from tensorflow.keras import layers, models, regularizers

/opt/conda/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:28: Depreca

In [2]:
MODEL_NAME = 'model09'
BUCKET = 'gs://etsuji-car-racing-v2'
os.environ['BUCKET'] = BUCKET
!gsutil mb -c regional -l us-west1 $BUCKET
!gsutil ls $BUCKET

Creating gs://etsuji-car-racing-v2/...
ServiceException: 409 A Cloud Storage bucket named 'etsuji-car-racing-v2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
gs://etsuji-car-racing-v2/model04/
gs://etsuji-car-racing-v2/model05/
gs://etsuji-car-racing-v2/model06/
gs://etsuji-car-racing-v2/model07/
gs://etsuji-car-racing-v2/model08/


In [3]:
class ApplySoftMaxWeight(layers.Layer):
    def __init__(self, **kwargs):
        self.filter_shape = None
        super(ApplySoftMaxWeight, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='weights', shape=[input_shape[3]])
        self.filter_shape = input_shape

    def get_config(self):
        config = super().get_config()
        return config
    
    def call(self, inputs, **kwargs):
        return inputs * tf.nn.softmax(self.kernel)
    
# Base model
class QValue:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        cnn_input = layers.Input(shape=(48, 48, 3), name='cnn_input')
        cnn1 = layers.Conv2D(16, (5, 5), padding='same',
                             use_bias=True, activation='relu',
                             name='cnn1')(cnn_input)
        pool1 = layers.MaxPooling2D((2, 2), name='pool1')(cnn1)
        cnn2 = layers.Conv2D(16, (5, 5), padding='same',
                             use_bias=True, activation='relu',
                             name='cnn2')(pool1)        
        pool2 = layers.MaxPooling2D((2, 2), name='pool2')(cnn2)
        weighted_filters = ApplySoftMaxWeight(name='weighted_filters')(pool2)

        cnn_flatten = layers.Flatten(name='flatten')(weighted_filters)
        action_input = layers.Input(shape=(5,), name='action_input')
        combined = layers.concatenate([cnn_flatten, action_input], name='concat')
        hidden1 = layers.Dense(2048, activation='relu', name='dense1')(combined)
        hidden2 = layers.Dense(1024, activation='relu', name='dense2')(hidden1)
        hidden3 = layers.Dense(512, activation='relu', name='dense3')(hidden2)
        q_value = layers.Dense(1, name='output')(hidden3)

        model = models.Model(inputs=[cnn_input, action_input], outputs=q_value)
        model.compile(loss='mse')
        return model

    def get_action(self, state):
        states = []
        actions = []
        for a in range(5):
            states.append(np.array(state))
            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
  
        q_values = self.model.predict([np.array(states), np.array(actions)])
        optimal_action = np.argmax(q_values)
        return optimal_action, q_values[optimal_action][0]

In [4]:
q_value = QValue()
q_value.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cnn_input (InputLayer)         [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 cnn1 (Conv2D)                  (None, 48, 48, 16)   1216        ['cnn_input[0][0]']              
                                                                                                  
 pool1 (MaxPooling2D)           (None, 24, 24, 16)   0           ['cnn1[0][0]']                   
                                                                                                  
 cnn2 (Conv2D)                  (None, 24, 24, 16)   6416        ['pool1[0][0]']                  
                                                                                              

2022-08-05 05:08:57.788088: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-05 05:08:57.788136: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-8-20220801-151053): /proc/driver/nvidia/version does not exist
2022-08-05 05:08:57.789541: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def join_frames(o0, o1, o2):
    gray_image0 = cv2.cvtColor(cv2.resize(o0, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image1 = cv2.cvtColor(cv2.resize(o1, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image2 = cv2.cvtColor(cv2.resize(o2, (48, 48)), cv2.COLOR_RGB2GRAY)
    
    return np.array(
        [gray_image0.transpose(),
         gray_image1.transpose(),
         gray_image2.transpose()]).transpose()

In [6]:
def get_episode(environ, q_value, epsilon):
    episode = []
    o0 = environ.reset()
    o1 = copy.deepcopy(o0)
    o2 = copy.deepcopy(o0)
    total_r = 0

    if epsilon > 0:
        keep_count = 3
    else:
        keep_count = 1

    c = 0
    while True:
        if c % keep_count == 0: # Get new action
            if np.random.random() < epsilon:
                a = np.random.randint(5)
            else:
                a, _ = q_value.get_action(join_frames(o0, o1, o2))
        c += 1
        o_new, r, done, inf = environ.step(a)                
        total_r += r

        # Terminate episode when total reward becomes negative
        if total_r < 0:
            done = 1

        if done:
            # Terminal state is to achive more than 990 or get out of the field.
            if total_r > 990 or r < -99:
                episode.append((join_frames(o0, o1, o2), a, r, None))
            break
        else:
            episode.append((join_frames(o0, o1, o2), a, r, join_frames(o1, o2, o_new)))
        o0, o1, o2 = o1, o2, o_new

    print('epsilon={}, episode length={}, total rewards={}'.format(epsilon, len(episode), total_r))
    return episode, total_r

In [7]:
def train(environ, q_value, epsilon, checkpoint=0):
    if checkpoint > 0:
        filename = 'car-racing-v2-{}-{}.hd5'.format(checkpoint, MODEL_NAME)
        subprocess.run(['gsutil', 'cp', '{}/{}/{}'.format(BUCKET, MODEL_NAME, filename), './'])
        print('load model {}'.format(filename))
        q_value.model = models.load_model(filename)
        os.remove(filename)

    experience = []
    good_experience = []
    best_r = [-100, -100, -100]

    for n in range(checkpoint + 1, checkpoint + 1000):
        print('iteration {}'.format(n))

        total_len = 0
        if n % 3 == 0:
            print('Testing the current performance...')
            episode, total_r = get_episode(environ, q_value, epsilon=0)
            with open('result.txt', 'a') as f:
                f.write('{},{},{},{}\n'.format(n, epsilon, len(episode), total_r))
            filename = 'car-racing-v2-{}-{}.hd5'.format(n, MODEL_NAME)
            q_value.model.save(filename, save_format='h5')
            subprocess.run(['gsutil', '-m', 'cp',
                            '{}'.format(filename), '{}/{}/'.format(BUCKET, MODEL_NAME)])
            os.remove(filename)
            experience += episode
            total_len += len(episode)

        while total_len < 500:
            episode, total_r = get_episode(environ, q_value, epsilon)
            total_len += len(episode)
            experience += episode

            # Keep the top 3 episodes
            if total_r > min(best_r):
                best_r = best_r[1:] + [total_r]
                good_experience += episode
                if len(good_experience) > 999 * 3:
                    good_experience = good_experience[-999 * 3:]

            
        if len(experience) > 999 * 5: # remember last 5 episodes
            experience = experience[-999 * 5:]

        epsilon = (epsilon - 0.2) * 0.99 + 0.2

        print('Training the model...')
        # Use latest episode + past episodes (sampling) + top 3 episode (sampling)
        latest_experience = experience[-total_len:]
        past_experience = experience[:-total_len]
        examples = latest_experience + \
            random.sample(past_experience, min(len(past_experience), 999)) + \
            random.sample(good_experience, min(len(good_experience), 999))
        
        # Show some statistics
        print('experience length={}'.format(len(experience)))
        print('number of examples={}'.format(len(examples)))
        print('best total reward = ', best_r)
        np.random.shuffle(examples)
                        
        states, actions, labels = [], [], []
        for state, a, r, state_new in examples:
            states.append(np.array(state))

            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
            
            if state_new is None:   # Terminal state
                q_new = 0
            else:
                _, q_new = q_value.get_action(state_new)
            labels.append(np.array(r + q_new))

        hist = q_value.model.fit(
            [np.array(states), np.array(actions)], np.array(labels),
            batch_size=50, epochs=10, verbose=0)
        print('loss = {}'.format(hist.history['loss']))

In [8]:
env = gym.make("CarRacing-v2", continuous=False)
q_value = QValue()
q_value.model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cnn_input (InputLayer)         [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 cnn1 (Conv2D)                  (None, 48, 48, 16)   1216        ['cnn_input[0][0]']              
                                                                                                  
 pool1 (MaxPooling2D)           (None, 24, 24, 16)   0           ['cnn1[0][0]']                   
                                                                                                  
 cnn2 (Conv2D)                  (None, 24, 24, 16)   6416        ['pool1[0][0]']                  
                                                                                            

/opt/conda/lib/python3.7/site-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/opt/conda/lib/python3.7/site-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


In [ ]:
train(env, q_value, epsilon=1.0, checkpoint=0)

iteration 1
epsilon=1.0, episode length=134, total rewards=-0.0771812080536709
epsilon=1.0, episode length=147, total rewards=-0.09411764705879855
epsilon=1.0, episode length=212, total rewards=-0.023404255319128903
epsilon=1.0, episode length=107, total rewards=-0.04731182795697572
Training the model...
experience length=600
number of examples=1200
best total reward =  [-0.09411764705879855, -0.023404255319128903, -0.04731182795697572]
loss = [4670.14404296875, 8.497062683105469, 6.520956039428711, 0.2938860058784485, 0.2749646306037903, 0.3223553001880646, 0.32706350088119507, 0.37113627791404724, 0.36666688323020935, 0.26872125267982483]
iteration 2
epsilon=0.992, episode length=187, total rewards=-0.07340823970033594
epsilon=0.992, episode length=186, total rewards=-0.06645962732914446
epsilon=0.992, episode length=107, total rewards=-0.08571428571427231
epsilon=0.992, episode length=135, total rewards=-0.08648648648645474
Training the model...
experience length=1215
number of exam

Copying file://car-racing-v2-3-model09.hd5 [Content-Type=application/octet-stream]...
- [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


epsilon=0.9840800000000001, episode length=101, total rewards=-0.06486486486484577
epsilon=0.9840800000000001, episode length=97, total rewards=-0.028013029315954813
epsilon=0.9840800000000001, episode length=179, total rewards=-0.014388489208591454
epsilon=0.9840800000000001, episode length=206, total rewards=-0.01034482758616237
Training the model...
experience length=1873
number of examples=2656
best total reward =  [-0.028013029315954813, -0.014388489208591454, -0.01034482758616237]
loss = [0.22956529259681702, 0.2536509037017822, 0.2236800193786621, 0.21344134211540222, 0.22508974373340607, 0.2061450034379959, 0.20475661754608154, 0.20286844670772552, 0.19038625061511993, 0.20572708547115326]
iteration 4
epsilon=0.9762392000000002, episode length=101, total rewards=-0.030508474576256023
epsilon=0.9762392000000002, episode length=98, total rewards=-0.06393442622948986
epsilon=0.9762392000000002, episode length=91, total rewards=-0.05365853658536346
epsilon=0.9762392000000002, episo

Copying file://car-racing-v2-6-model09.hd5 [Content-Type=application/octet-stream]...
- [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


epsilon=0.9607920399200003, episode length=326, total rewards=-0.020261437908432017
epsilon=0.9607920399200003, episode length=114, total rewards=-0.005747126436758537
Training the model...
experience length=3749
number of examples=2570
best total reward =  [-0.01034482758616237, -0.012987012987008856, -0.005747126436758537]
loss = [0.2860679030418396, 0.23285672068595886, 0.2230360209941864, 0.21395818889141083, 0.20462679862976074, 0.1867469996213913, 0.1832372099161148, 0.1872856467962265, 0.17411649227142334, 0.16755561530590057]
iteration 7
epsilon=0.9531841195208004, episode length=221, total rewards=-0.04810126582273597
epsilon=0.9531841195208004, episode length=132, total rewards=-0.09867986798679174
epsilon=0.9531841195208004, episode length=170, total rewards=-0.03515358361771856
Training the model...
experience length=4272
number of examples=2521
best total reward =  [-0.01034482758616237, -0.012987012987008856, -0.005747126436758537]
loss = [0.3321106731891632, 0.2796126008

...

Training the model...
experience length=4995
number of examples=2997
best total reward =  [339.73941368078044, 292.9824561403516, 308.9219330855014]
loss = [4.170889854431152, 3.895979642868042, 3.8873653411865234, 3.714294672012329, 3.411482095718384, 3.465815782546997, 3.3674840927124023, 3.4426867961883545, 3.218010663986206, 3.172297239303589]
iteration 379
epsilon=0.21791404021682076, episode length=999, total rewards=215.9509202454019
Training the model...
experience length=4995
number of examples=2997
best total reward =  [339.73941368078044, 292.9824561403516, 308.9219330855014]
loss = [3.679853916168213, 3.2255191802978516, 3.4825246334075928, 3.3809072971343994, 3.1890628337860107, 3.37123703956604, 3.1167380809783936, 3.103140354156494, 3.216972589492798, 3.1255502700805664]
iteration 380
epsilon=0.21773489981465255, episode length=999, total rewards=267.0411985018743
Training the model...
experience length=4995
number of examples=2997
best total reward =  [339.7394136807804

Copying file://car-racing-v2-381-model09.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


Training the model...
experience length=4995
number of examples=2965
best total reward =  [339.73941368078044, 292.9824561403516, 308.9219330855014]
loss = [46.046512603759766, 44.24314880371094, 42.89369583129883, 42.594547271728516, 39.97822570800781, 40.68159103393555, 38.44552230834961, 37.011940002441406, 37.80597686767578, 35.969478607177734]
iteration 382
epsilon=0.21738197530834097, episode length=999, total rewards=278.9473684210539
Training the model...
experience length=4995
number of examples=2997
best total reward =  [339.73941368078044, 292.9824561403516, 308.9219330855014]
loss = [3.394462823867798, 3.493769884109497, 3.382167339324951, 3.103400707244873, 3.2903382778167725, 3.244814395904541, 3.189795970916748, 3.169276714324951, 3.192811965942383, 3.114851713180542]
iteration 383
epsilon=0.21720815555525755, episode length=999, total rewards=123.21428571428689
Training the model...
experience length=4995
number of examples=2997
best total reward =  [339.73941368078044,

Copying file://car-racing-v2-384-model09.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


Training the model...
experience length=4995
number of examples=2997
best total reward =  [339.73941368078044, 292.9824561403516, 308.9219330855014]
loss = [7.521159648895264, 4.877452850341797, 5.310811996459961, 4.112927436828613, 5.603460311889648, 4.059048175811768, 4.192379951477051, 4.259861946105957, 4.3823561668396, 3.938472270965576]
iteration 385
epsilon=0.21686571325970794, episode length=999, total rewards=32.352941176472974
Training the model...
experience length=4995
number of examples=2997
best total reward =  [339.73941368078044, 292.9824561403516, 308.9219330855014]
loss = [3.434981107711792, 2.984114646911621, 3.1241986751556396, 3.0823638439178467, 2.934863567352295, 2.7383484840393066, 2.8384063243865967, 2.7126660346984863, 2.7843167781829834, 2.849518060684204]
iteration 386
epsilon=0.21669705612711085, episode length=999, total rewards=492.30769230768504
Training the model...
experience length=4995
number of examples=2997
best total reward =  [292.9824561403516, 